In [2]:
from data_processing_try import *
from train_trace_2 import *
from visualization import plot_ith_segment
import pandas as pd
import tensorflow as tf

In [3]:
#data = Dataset("set1_imu.csv", "set1_tab.csv")
#data.data_set.to_csv('data/data_frame_smooth.csv')  

In [4]:
data = pd.read_csv(".\data\data_frame_smooth.csv")

In [5]:
data.head()

,Unnamed: 0,t_r,x,y,z,ax,ay,az,gx,gy,...,r,nav_ax,nav_ay,nav_az,vel_x,vel_y,vel_z,pos_x,pos_y,pos_z
0,0,0.0,171.081563,175.444042,0.0,-0.846652,0.333538,0.358858,0.030683,-0.051090,...,[[ 0.36171091 -0.16438019 -0.91768425]\n [ 0.6...,-0.690389,-0.256070,-0.643924,0.000000,0.000000,0.000000,171.081563,175.444042,0.000000
1,1,0.5,170.148674,174.619624,0.0,-0.863770,0.266961,0.328226,-0.098304,-0.025454,...,[[ 0.26588484 -0.26319444 -0.92738015]\n [ 0.6...,-0.604316,-0.354587,-0.658915,-0.323676,-0.152664,-0.325710,171.000643,175.405875,-0.081427
2,2,1.0,169.290593,173.889512,0.0,-0.878953,0.216098,0.305050,-0.191155,-0.007150,...,[[ 0.24343795 -0.31012972 -0.91899811]\n [ 0.6...,-0.561329,-0.417381,-0.650395,-0.615088,-0.345656,-0.653037,170.765953,175.281295,-0.326114
3,3,1.5,168.504520,173.249715,0.0,-0.882562,0.259232,0.326468,-0.067845,-0.032703,...,[[ 0.2665411 -0.31219316 -0.91186143]\n [ 0.6...,-0.613863,-0.398371,-0.645887,-0.908885,-0.549594,-0.977108,170.384959,175.057483,-0.733650
4,4,2.0,167.787657,172.696245,0.0,-0.885694,0.302587,0.341227,0.028487,-0.045747,...,[[ 0.29297023 -0.29502853 -0.90946501]\n [ 0.6...,-0.659088,-0.375734,-0.645659,-1.227123,-0.743121,-1.299994,169.850957,174.734304,-1.302926


In [6]:
#plot_ith_segment(data, 10)

In [7]:
data.columns

Index(['Unnamed: 0', 't_r', 'x', 'y', 'z', 'ax', 'ay', 'az', 'gx', 'gy', 'gz',
       'segment', 'q0', 'q1', 'q2', 'q3', 'r', 'nav_ax', 'nav_ay', 'nav_az',
       'vel_x', 'vel_y', 'vel_z', 'pos_x', 'pos_y', 'pos_z'],
      dtype='object')

In [10]:
df = data.drop(['t_r', 'x', 'y', 'z','segment','r'], axis=1)
pos = data.loc[:,['x', 'y', 'z']]

In [11]:
df.head()

,Unnamed: 0,ax,ay,az,gx,gy,gz,q0,q1,q2,q3,nav_ax,nav_ay,nav_az,vel_x,vel_y,vel_z,pos_x,pos_y,pos_z
0,0,-0.846652,0.333538,0.358858,0.030683,-0.051090,-0.443710,0.789048,0.241368,0.502618,-0.257913,-0.690389,-0.256070,-0.643924,0.000000,0.000000,0.000000,171.081563,175.444042,0.000000
1,1,-0.863770,0.266961,0.328226,-0.098304,-0.025454,-0.393085,0.776182,0.174596,0.532326,-0.289287,-0.604316,-0.354587,-0.658915,-0.323676,-0.152664,-0.325710,171.000643,175.405875,-0.081427
2,2,-0.878953,0.216098,0.305050,-0.191155,-0.007150,-0.326313,0.773983,0.150563,0.534482,-0.304320,-0.561329,-0.417381,-0.650395,-0.615088,-0.345656,-0.653037,170.765953,175.281295,-0.326114
3,3,-0.882562,0.259232,0.326468,-0.067845,-0.032703,-0.162954,0.779306,0.161099,0.521361,-0.308078,-0.613863,-0.398371,-0.645887,-0.908885,-0.549594,-0.977108,170.384959,175.057483,-0.733650
4,4,-0.885694,0.302587,0.341227,0.028487,-0.045747,-0.035403,0.782853,0.183377,0.508810,-0.307616,-0.659088,-0.375734,-0.645659,-1.227123,-0.743121,-1.299994,169.850957,174.734304,-1.302926


In [ ]:
df["previous_1"] = 

In [12]:
pos.head()

,x,y,z
0,171.081563,175.444042,0.0
1,170.148674,174.619624,0.0
2,169.290593,173.889512,0.0
3,168.504520,173.249715,0.0
4,167.787657,172.696245,0.0


In [14]:
split = int(len(df) * 0.8)

features = df.to_numpy()
f_train = features[:split, :] #(164658, 20)
f_test = features[split:,:] # shape (41165, 20)


p = pos.to_numpy()
p_train = p[:split, :] #(164658, 3)
p_test = p[split:, :]

In [19]:
time_series = tf.keras.utils.timeseries_dataset_from_array(f_train, p_train, sequence_length=10)

In [20]:
time_series.shape

AttributeError: 'BatchDataset' object has no attribute 'shape'